In [11]:
import json
import re
import requests
import csv
import pandas as pd
import time
from ast import literal_eval
import urllib
from urllib.parse import quote

with open('./top-level-lcc.json', 'r') as reader:
    basic = json.load(reader)

with open('./v2_merged_lcc.json', 'r') as reader: #using the version with a lot of detail for black history and lgbtq history
    detail = json.load(reader)

syllabus = pd.read_csv('./edstud_lgbt_med.csv')
syllabuslow = pd.read_csv('./edstud_lgbt_low.csv')
syllabushigh = pd.read_csv('./edstud_lgbt_high.csv')

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [2]:
def split_lcc(call_number):
    if call_number == None:
        return None
    #specifically for how OL formats their LCC call numbers
    out = call_number.replace('-', '') #remove hyphen
    out = re.sub(r'(?<=[A-Z])0+', '', out) #Remove leading zeros after the letters segment
    
    # Adjust regex match for letter and number segments, ensuring float conversion for fractional part
    match = re.match(r'([A-Z]+)(\d+(\.\d+)?)', out)
    
    if match:
        return (match.group(1), float(match.group(2)))
    else:
        return None

def lookup_meaning(code): #takes in tuple (call number)
    l = []

    try:
        d = detail[code[0]]
        for i in d:
            if code[1] >= i['start'] and code[1] <= i['stop']:
                l.append(i['subject'])
    except:
        pass

    return l
    #returns a list of definitions for the code

In [5]:
ex1 = 'HV-1568.00000000.B376 2016' #The Minority Body by Elizabeth Barnes
ex2 = 'DAW1008.00000000.B37 1987' #A guide to Central Europe by Richard Bassett
ex3 = 'E--185.86.00000000.H739 2001' #Salvation: Black People and Love by bell hooks
ex4 = 'PL-0788.40000000.G4 E5 2000' #Genji Monogatari by Murasaki Shikibu
ex5 = 'DS-0032.80000000' #Orientalism by Edward Said

print(lookup_meaning(split_lcc(ex1)))
print(lookup_meaning(split_lcc(ex2)))
print(lookup_meaning(split_lcc(ex3)))
print(lookup_meaning(split_lcc(ex4)))
print(lookup_meaning(split_lcc(ex5)))

['Special classes', 'People with disabilities', 'Protection, assistance and relief', 'Social pathology.  Social and public welfare.']
['General', 'History of Central Europe']
['Afro-Americans', 'Status and development since emancipation', 'United States', 'Elements in the population', 'United States. Elements in the population. African Americans. Social conditions. Social life and customs.']
['Languages of Eastern Asia, Africa, Oceania', 'Individual authors and works', 'Japanese language and literature', 'Japanese literature']
['History', 'History of Asia']


Search by LCCN or ISBN on OpenLibrary:

In [3]:
def searchby_lccn(lccn, fields = 'author_name,subject,lcc,title', limit = 5): 
    r = []
    response = requests.get(f'https://openlibrary.org/search.json?q=lcc:{lccn}&fields={fields}&limit={limit}').json()
    return response['docs']

def searchby_isbn(isbn, field = 'lcc', limit = 1):
    time.sleep(2) #being polite
    response = requests.get(f'https://openlibrary.org/search.json?q=isbn:{isbn}&fields={field}&limit={limit}').json()

    if bool(response['docs']): #falsy
        #print(response['docs'], isbn) #error checking

        if bool(response['docs'][0].get(f'{field}')): #if there is an lcc
            return response['docs'][0].get(f'{field}')[0] #string, first lcc returned
    else:
        return '' #nothing returned

def reformat_openlibrary_lccn(syllabus): #doesn't account for specific subclasses
    lccn_tup = []

    for isbn in syllabus['isbn']: #get the lccn
        val = split_lcc(searchby_isbn(isbn)) #after querying Open library, split them into tuples

        if val is not None:
            lccn_tup.append(val)
            
    return lccn_tup

In [6]:
#queries
#1. Direct call number: HV3181*
print('Search by call number:', searchby_lccn('HV3181\*', 'title, lcc', 1))

#2. Anything under a broad category: H* includes H, HA, HB, etc.
print('Search by class:', searchby_lccn('H\*', 'title', 1))

#3. Anything under a specific category: H-- includes only H, HA- includes only HA
print('Search by Subclass:', searchby_lccn('H--\*', 'title', 1))
#books might differ slightly in exact code, so i limit to only title here, you can look at the lcc return list for more details

#4. A range of outputs: HQ TO HV
print('Search range:', searchby_lccn('[HQ TO HV]', 'title', 1)) #can also do this with numbers

#5. A string of call numbers is not allowed to my understanding
#search_lccn('[HJ6799, HJ8001]', 'title,lcc', 2)

#this could be made simpler; right now i return a tuple for simplification when looking up keys in the dictionary
print("Search with Example 2:", searchby_lccn(split_lcc(ex2)[0] + str(split_lcc(ex2)[1]) +'*', 'subject', 1))

Search by call number: [{'lcc': ['HV-3181.00000000.S64', 'HV-3181.00000000'], 'title': 'Black Empowerment'}]
Search by class: [{'title': 'The Roman Breviary'}]
Search by Subclass: [{'title': 'The Roman Breviary'}]
Search range: [{'title': 'The Canterbury Tales'}]
Search with Example 2: [{'subject': ['Guidebooks', 'Travel & holiday guides', 'Travel Guides', 'Travel', 'Travel - Foreign', 'Central Europe', 'Europe - General']}]


In [9]:
lccn_tup = reformat_openlibrary_lccn(syllabus)
print(lccn_tup)

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/Users/winiboya/opt/anaconda3/lib/python3.9/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1949, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/Users/winiboya/opt/anaconda3/lib/python3.9/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1984, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

Find common parents in order to make the most efficient search

In [7]:
def get_all_parents(lccn, lcc_data):
    init = lccn[0] + str(lccn[1])
    all_parents = {init} #let itself be a "parent" just in case!

    try:
        d = lcc_data[lccn[0]] #key is first element in list

        for i in d: #for each dictionary in the list
            if lccn[1] >= i['start'] and lccn[1] <= i['stop']: #check if a subset

                #if this is the deepest node; if itself is the only parent, until now, then overwrite
                if len(i['parents']) >= len(all_parents): 
                    all_parents = i['parents']
        return all_parents
    except:
        return None


def find_most_recent_common_parent(tupes, lcc_data):
    node_parent_sets = [get_all_parents(t, lcc_data) for t in tupes]
    
    prefixes = {}
    inter = {}
    #get all parents for each prefix
    for t in tupes:
        val = get_all_parents(t, lcc_data)
        if val != None:
            if t[0] not in prefixes.keys():
                prefixes[t[0]] = [val] #make a list with all the floats
            else:
                prefixes[t[0]].append(val) #make a list with all the floats

    for k,v in prefixes.items():
        inter[k] = list(set(v[0]).intersection(*map(set, v[1:])))[-1] #make it a string, choose most specific one

    return inter

In [10]:
mrcp = find_most_recent_common_parent(lccn_tup, detail)
print(mrcp)

{'HQ': 'HQ1-2044', 'LB': 'LB5-3640', 'LC': 'LC8-6691', 'BF': 'BF1-990', 'PZ': 'PZ1-90'}


Our area of desired diversity (perspective of interest in other words) is LGBTQ Studies. Our overarching discipline area is Education.

Could this level of granularity be useful? Unsure

In [11]:
with open('./updated_lgbtq_lcc.json', 'r') as reader: #everything about lgbtq studies, specifically
    diversity = json.load(reader) #in practice, we call also ONLY load in the relevant subclasses

In [12]:
def find_diversity_topics(syll):
    #vals = []
    topics = []
    
    mrcp = find_most_recent_common_parent(reformat_openlibrary_lccn(syll), detail)
    div_dict = {k: v for k,v in diversity.items() if k in mrcp.keys()}

    for k,v in mrcp.items():
        try:
            entries = div_dict[k]
            #vals.append({k: [entry for entry in entries if v in entry['parents']]}) #gives a lot of topics underneath the parent node
            topics.append([entry['subject'] for entry in entries if v in entry['parents']])

        except:
            pass
    return topics

topics = find_diversity_topics(syllabus)
print(topics)

[['Human sexuality. Sex. Sexual orientation.', 'Kinsey, Alfred.', 'Bisexuality. General works.', 'Bisexuality. By region or country, A-Z.', 'Homosexuality. Lesbianism. Periodicals. Serials.', 'Homosexuality. Lesbianism. Congresses.', 'Homosexuality. Lesbianism. Societies.', 'Homosexuality. Lesbianism. Dictionaries.', 'Homosexuality. Lesbianism. Computer networks. Electronic information resources (including the Internet and digital libraries).', 'Gay and lesbian studies.', 'Homosexuality. Lesbianism. Biography (Collective).', 'Homosexuality. Lesbianism. Travel.', 'Homosexuality. Lesbianism. Gay parents.', 'Lesbians. Biography. Collective.', 'Lesbians. Biography. Individual, A-Z.', 'Lesbians. General works.', 'Lesbians. Sex instruction.', 'Lesbian mothers.', 'Middle-aged lesbians. Older lesbians.', 'Lesbians. By region or country, A-Z.', 'Gay men. Biography. Collective.', 'Gay men. Biography. Individual, A-Z.', 'Kameny, Frank.', 'Gay men. General works.', 'Gay men. Sex instruction.', 'Ga

Getting the distribution of key words

In [13]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords') #to remove uninformative words
stop_words = set(stopwords.words('english'))
lcc_stop = open("lcc_stop_words.txt", "r").read().split("\n") #stop words for the library of congress classification system

In [14]:

def get_prop_occurrences(topics_lst, lcc_stop = lcc_stop, kind = 'by phrase', top_n = 15): #splits by phrase/full subject
    all_tags = []

    if type(topics_lst[0]) == str:
        topics_lst = [topics_lst] #kinda nasty srry, but its how it functions

    if kind == 'by phrase':
        for i in topics_lst:
            tags = '. '.join(i).split('. ')
            tags = [x.lower().split('.')[0] for x in tags]
            tags = [x for x in tags if not any(sub in x for sub in lcc_stop)] #get rid of common but uninformative loc terms
            tags = [' '.join([word for word in x.split(' ') if word not in stop_words]) for x in tags] #get rid of stop words
            tags = [x.lstrip().rstrip() for x in tags] #remove leading and trailing ws

            all_tags += tags

    else: #if by words
        for i in topics_lst:
            tags = ' '.join(i).split() #split by words
            tags = [x.lower().split('.')[0].split(',')[0].split(')')[0] for x in tags] #pruning for commas, periods, and parenthesis
            tags = [x.lstrip().rstrip() for x in tags] 
            tags = [x for x in tags if x not in lcc_stop or x not in stop_words] #words that I don't want are appearing for some reason
            print(tags)

            all_tags += tags

    #make proportions
    prop = Counter(all_tags) 
    prop = dict(prop.most_common(top_n))
    total = sum(prop.values())
    prop = {k: v/total for k, v in prop.items()}
    return prop

In [15]:
prop_div = get_prop_occurrences(topics)
print(prop_div)
print(get_prop_occurrences(topics, kind = 'by words'))

{'homosexuality': 0.15714285714285714, 'lesbianism': 0.15714285714285714, 'biography': 0.11428571428571428, 'gay lesbian culture': 0.1, 'lesbians': 0.07142857142857142, 'gay men': 0.07142857142857142, 'transvestism': 0.05714285714285714, 'transsexualism': 0.05714285714285714, 'homophobia': 0.04285714285714286, 'bisexuality': 0.02857142857142857, 'sex instruction': 0.02857142857142857, 'same-sex relationships': 0.02857142857142857, 'heterosexism': 0.02857142857142857, 'gay rights movement': 0.02857142857142857, 'gay liberation movement': 0.02857142857142857}
['human', 'sexuality', 'sex', 'sexual', 'orientation', 'kinsey', 'alfred', 'bisexuality', 'general', 'works', 'bisexuality', 'by', 'region', 'or', 'country', 'a-z', 'homosexuality', 'lesbianism', 'periodicals', 'serials', 'homosexuality', 'lesbianism', 'congresses', 'homosexuality', 'lesbianism', 'societies', 'homosexuality', 'lesbianism', 'dictionaries', 'homosexuality', 'lesbianism', 'computer', 'networks', 'electronic', 'informat

Get the same distribution of topics for books on the syllabus

In [141]:
topics_syll = []

for i in lccn_tup:
   topics_syll += lookup_meaning(i)

print(topics_syll)
prop_syll = get_prop_occurrences(topics_syll)
print(prop_syll)

['Sex role', 'The Family.  Marriage.  Women', 'Sex role', 'The Family.  Marriage.  Women', 'Early childhood education', 'Theory and practice of education', 'Theory and practice of education', 'Elementary or public school education', 'Preschool education.    Nursery schools', 'Theory and practice of education', 'Early childhood education', 'Theory and practice of education', 'Community and the school', 'Home and school', 'Special aspects of education', 'Social aspects of education', 'School management and discipline', 'Theory and practice of education', 'School administration and organization', 'Consciousness.  Cognition', 'Psychology', 'Fiction and juvenile belles lettres', 'Juvenile belles lettres', 'Multicultural education (General)', 'Special aspects of education', 'Types of education', 'The Family.  Marriage.  Women', 'Children.  Child development', 'The family.  Marriage.  Home']
{'family': 0.16666666666666666, 'marriage': 0.16666666666666666, 'women': 0.125, 'sex role': 0.0833333

Looking up books in the range with a set list of important topics

In [63]:
def search_subjects(lcc, topics = [], discipline_tags = [], diversity_tags = [], field = 'subject', limit = 1, exact_string_matching = False):
    if type(topics) == str:
        time.sleep(2) #being polite
        response = requests.get(f'https://openlibrary.org/search.json?q=lcc:{lcc}&subject={topics}&fields={field}&limit={limit}').json()

        if bool(response['docs']): #falsy
            return response['docs']
        else:
            return '' #nothing returned

    elif bool(topics):    
        q = f'https://openlibrary.org/search.json?q=lcc:{lcc}&fields={field}&limit={limit}&subject:'

        if exact_string_matching: #for cases where a single word is used
            topics = list(map(lambda x: f"\"{x}\"", topics)) #exact string matching

            
        topics = list(map(lambda x: urllib.parse.quote(x.encode("utf-8")), topics)) #encode tags

        topics = '+OR+subject:'.join(topics) #comma (,) and pipe (|) are similar AND, not OR for some reason
        #topics = ''.join(list(map(lambda x: f'&subject={x}', topics)))
        #topics = ','.join(topics)

        q += topics
        
        print(q)

        time.sleep(2) #being polite
        response = requests.get(q).json()
        
        if bool(response['docs']): #falsy
            return response['docs']
        else:
            return '' #nothing returned

    elif bool(discipline_tags) and bool(diversity_tags):
        #encode URI
        discipline_tags, diversity_tags = list(map(lambda x: urllib.parse.quote(x.encode("utf-8")), discipline_tags)), list(map(lambda x: urllib.parse.quote(x.encode("utf-8")), diversity_tags))
        #if this ever throws errors, maybe we need to specify unicode

        if exact_string_matching: #for cases where results by word is used
            discipline_tags, diversity_tags = list(map(lambda x: f"\"{x}\"", discipline_tags)), list(map(lambda x: f"\"{x}\"", diversity_tags))

        #match any of the tags
        str_disc, str_div = '+OR+subject:'.join(discipline_tags), '+OR+subject:'.join(diversity_tags)
        q = f'https://openlibrary.org/search.json?q=lcc:{lcc} AND ((subject:{str_disc}) AND (subject:{str_div}))&fields={field}&limit={limit}'
        
        print(q)
        response = requests.get(q).json()
        
        if bool(response['docs']): #falsy
            return response['docs']
        else:
            return '' #nothing returned
            
    else:
        return None

Use most recent common parents as a potential narrower

Trying Rao's Entropy!

In [ ]:
import pandas as pd
import numpy as np
import tensorflow_hub
from sentence_transformers import SentenceTransformer #"tensorflow>=1.7.0", tensorflow-hub

In [ ]:
# Load model for embeddings
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = tensorflow_hub.load(module_url)

print ("module %s loaded" % module_url)

def embed(input):
  return model(input)

This is not a measure of entropy over the syllabus, this is a comparison between the syllabus and the diversity area topics

In [150]:
# Load pre-trained model for sentence embeddings

# Define Rao's entropy formula
def raos_entropy(all_cats):
    #i'm aware this is presently incorrect bc the probably of topics is not btwn 0 and 1, but This Is a Start!

    # Calculate pairwise cosine distances between topics
    tags = list(all_cats.keys())
    embeddings = embed(tags) #needs to be embedded over one space
    distance_matrix = np.inner(embeddings, embeddings) #cosine sim

    # rao's entropy
    rqe = 0.0

    for i, cat_i in enumerate(tags):
        for j, cat_j in enumerate(tags):
            p_i = all_cats.get(cat_i, 0) # Probability for category i (fall through if 0)
            p_j = all_cats.get(cat_j, 0)
            #print(p_i, p_j)
            # cosine distance (1 - cosine similarity)
            distance = 1 - distance_matrix[i, j]
            #print(distance)
            
            rqe += p_i * p_j * distance

    return rqe/2 #because you're summing the pairs twice

In [132]:
# Calculate diversity using Rao's entropy, medium syllabus
topics_syll = []

#for i in reformat_openlibrary_lccn(syllabus):
#   topics_syll += lookup_meaning(i)
for i in lccn_tup:
   topics_syll += lookup_meaning(i)
prop_syll = get_prop_occurrences(topics_syll, 'by word', 10) #by phrase isn't working at the moment
prop_div = get_prop_occurrences(find_diversity_topics(syllabus), 'by word', 10)

subject_terms = list(prop_div.keys()) + list(prop_syll.keys())

all_cats = {**prop_syll, **prop_div}
entropy = raos_entropy(all_cats)
print(f"Rao's Entropy (Diversity): {entropy}")

Rao's Entropy (Diversity): 2.6499308377215414


In [ ]:
topics_syll = []

for i in reformat_openlibrary_lccn(syllabuslow): #low diversity
   topics_syll += lookup_meaning(i)

print(topics_syll)

prop_syll = get_prop_occurrences(topics_syll, 'by word', 10)
prop_div = get_prop_occurrences(find_diversity_topics(syllabuslow), 'by word', 10)

all_cats = {**prop_syll, **prop_div}
entropy = raos_entropy(all_cats)
print(f"Rao's Entropy (Diversity): {entropy}")

['Early childhood education', 'Theory and practice of education', 'Theory and practice of education', 'Elementary or public school education', 'Preschool education.    Nursery schools', 'Theory and practice of education', 'Early childhood education', 'Theory and practice of education', 'School management and discipline', 'Theory and practice of education', 'School administration and organization', 'Consciousness.  Cognition', 'Psychology', 'Fiction and juvenile belles lettres', 'Juvenile belles lettres', 'Multicultural education (General)', 'Special aspects of education', 'Types of education', 'The Family.  Marriage.  Women', 'Children.  Child development', 'The family.  Marriage.  Home']
Rao's Entropy (Diversity): 2.783370945261346


In [151]:
topics_syll = []

for i in reformat_openlibrary_lccn(syllabushigh): #high diversity
   topics_syll += lookup_meaning(i)

print(topics_syll)

prop_syll = get_prop_occurrences(topics_syll, 'by word', 10)
prop_div = get_prop_occurrences(find_diversity_topics(syllabus), 'by word', 10)

all_cats = {**prop_syll, **prop_div}
entropy = raos_entropy(all_cats)
print(f"Rao's Entropy (Diversity): {entropy}")

['Sex role', 'The Family.  Marriage.  Women', 'Sex role', 'The Family.  Marriage.  Women', 'Early childhood education', 'Theory and practice of education', 'Theory and practice of education', 'Elementary or public school education', 'Preschool education.    Nursery schools', 'Theory and practice of education', 'Early childhood education', 'Theory and practice of education', 'Community and the school', 'Home and school', 'Special aspects of education', 'Social aspects of education', 'School management and discipline', 'Theory and practice of education', 'School administration and organization', 'Consciousness.  Cognition', 'Psychology', 'Fiction and juvenile belles lettres', 'Juvenile belles lettres', 'Multicultural education (General)', 'Special aspects of education', 'Types of education', 'The Family.  Marriage.  Women', 'Children.  Child development', 'The family.  Marriage.  Home', 'The Family.  Marriage.  Women', 'Sexual life', 'Educational sociology', 'Special aspects of education

To be implemented: RQE for set of book recommendations!

In [ ]:
search_subjects(lccn_query, discipline_tags = list(prop_syll.keys()), diversity_tags = list(prop_div.keys()), field = 'title,subject,isbn,author_name', limit = 3, exact_string_matching=True)

Simplify suggestion information

In [ ]:
def get_suggestions(mrcp, syll_topics, diversity_topics):
    suggestions = []
    for k,v in mrcp.items(): 
        if '-' in v:
            lst = v.split('-')
            lccn_query = '[' + lst[0] + ' TO ' + k + lst[1] + ']'
        suggestions += search_subjects(lccn_query, discipline_tags = syll_topics, diversity_tags = diversity_topics, field = 'title,subject,isbn,author_name', limit = 50, exact_string_matching=True)
        #list of author names, ISBNs, titles, subjects
    
    for book in suggestions:
        try:
            #print(next((i for i in book['isbn'] if re.match(r'^(979|978)\d{10}$', i)), None))
            book['isbn'] = next((i for i in book['isbn'] if re.match(r'^(979|978)\d{10}$', i)), None) #shorten the list of ISBNs, ISBN-13
        except:
            #print(book)
            book.update({'isbn': ''})
        book['subject'] = [x.lower() for x in book['subject']]
        #book['subject'] = [sub for sub in [x for x in book['subject']] if sub in syll_topics or sub in diversity_topics] #not working
        #book['subject'] = [x for x in book['subject'] if any(sub in x for sub in syll_topics) or any(sub in x for sub in diversity_topics)] #if it doesn't contain the tags for whatever reason
        book['subject'] = [sub for sub in syll_topics if any(sub in x for x in book['subject'])]
        book['subject'] += [sub for sub in diversity_topics if any(sub in x for x in book['subject'])]

    return suggestions

In [134]:
get_suggestions(mrcp, list(prop_syll.keys()), list(prop_div.keys()))

https://openlibrary.org/search.json?q=lcc:[HQ1 TO HQ2044] AND ((subject:education+OR+subject:theory+OR+subject:practice+OR+subject:school+OR+subject:family+OR+subject:marriage+OR+subject:women+OR+subject:aspects+OR+subject:sex+OR+subject:role+OR+subject:early+OR+subject:childhood+OR+subject:home+OR+subject:special+OR+subject:juvenile) AND (subject:gay+OR+subject:a-z+OR+subject:works+OR+subject:general+OR+subject:region+OR+subject:country+OR+subject:homosexuality+OR+subject:lesbianism+OR+subject:lesbian+OR+subject:biography+OR+subject:lesbians+OR+subject:men+OR+subject:culture+OR+subject:special+OR+subject:movement))&fields=title,subject,isbn,author_name&limit=50
https://openlibrary.org/search.json?q=lcc:[LB5 TO LB3640] AND ((subject:education+OR+subject:theory+OR+subject:practice+OR+subject:school+OR+subject:family+OR+subject:marriage+OR+subject:women+OR+subject:aspects+OR+subject:sex+OR+subject:role+OR+subject:early+OR+subject:childhood+OR+subject:home+OR+subject:special+OR+subject:ju

KeyboardInterrupt: 

In [105]:
suggestions

[{'author_name': ['Charlotte Brontë'],
  'isbn': '9780342301812',
  'title': 'Jane Eyre',
  'subject': ['Rape victims', 'Women, biography', 'Feminism']},
 {'author_name': ['Mark Twain'],
  'isbn': '9781096872146',
  'title': 'The Prince and the Pauper',
  'subject': ['Readers',
   'Fiction',
   'Poor',
   'Kings and rulers',
   'Impostors and imposture',
   'Poor children',
   'Boys',
   'Classic Literature',
   'Princes',
   'Adventure and adventurers',
   'Open Library Staff Picks',
   'Mistaken identity',
   'Lookalikes',
   'Adventure stories',
   'Social classes',
   'History',
   'Juvenile fiction',
   'Large print',
   'Sexual ethics for teenagers',
   'Sex instruction for youth',
   'Sexual ethics',
   'Large type books',
   "Children's fiction",
   'Princes, fiction',
   'Edward vi, king of england, 1537-1553, fiction',
   'Adventure and adventurers, fiction',
   'Boys, fiction',
   'England, fiction',
   'American fiction (fictional works by one author)',
   'London (england

select for maximal entropy

In [ ]:
for book in suggestions: #output of get_suggestions
    